In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import warnings
import scarches as sca
warnings.filterwarnings("ignore")


import sys
sys.path.append('../scripts')
%load_ext autoreload
%autoreload 2
#%load_ext lab_black

In [ ]:
adata = sc.read_h5ad('/mnt/storage/Daniele/atlases/mouse/14_mouse_final_annotation.h5ad')

In [ ]:
adata_manual = adata[:, adata.var['manual_gene']].copy()

In [ ]:
batch_key = 'donor_id'
celltype_key = 'Level_4_knn'

In [ ]:
# hotfix
adata_manual.obs[batch_key] = adata_manual.obs[batch_key].astype(str).astype('category')
sca.models.SCVI.setup_anndata(adata_manual, layer='binned_data', batch_key=batch_key, labels_key=celltype_key)


In [ ]:
vae = sca.models.SCVI(
    adata_manual,
    n_layers=2,
    encode_covariates=True,
    deeply_inject_covariates=False,
    use_layer_norm="both",
    use_batch_norm="none",
)

In [ ]:
vae.train(max_epochs=50)

In [ ]:
scanvae = sca.models.SCANVI.from_scvi_model(vae, unlabeled_category = "Unknown")
scanvae.train(max_epochs=10)

In [ ]:
adata_manual.obs['predictions'] = scanvae.predict()
print("Acc: {}".format(np.mean(adata_manual.obs.predictions == adata_manual.obs.Level_4_knn)))

In [ ]:
adata_manual.obsm['scANVI_emb_final'] = scanvae.get_latent_representation(adata_manual)

In [ ]:
adata.obsm['scANVI_emb_final'] = adata_manual.obsm['scANVI_emb_final'].copy()

In [ ]:
from sklearn_ann.kneighbors.annoy import AnnoyTransformer
sc.pp.neighbors(adata, transformer=AnnoyTransformer(15), use_rep='scANVI_emb_final')

In [ ]:
sc.tl.umap(adata, min_dist=0.25)

In [ ]:
adata.write_h5ad('/mnt/storage/Daniele/atlases/mouse/15_mouse_final_integration.h5ad')

# create clean low level annotations

In [ ]:
adata.obs.Level_4_knn.replace('Malignant Cell - Hihgly Invasive', 'Malignant Cell - Highly Invasive', inplace=True)
adata.obs.Level_4_knn.replace('Acinar idlling', 'Acinar Idling', inplace=True)

In [ ]:
adata.obs['Level_4_final'] = adata.obs['Level_4_knn'].copy()

In [ ]:
level4_to_level3 = {
    'Macrophage - M2-like TAM':            'Macrophage',
    'Macrophage - M1-like TAM':            'Macrophage',
    'Macrophage - lipid processing TAM':   'Macrophage',
    'Macrophage - angiogenic TAM':         'Macrophage',
    'Macrophage - CD3+ TAM':               'Macrophage',

    'Monocyte':                            'Monocyte',

    'B Cell - Naive':                      'B Cell',
    'B Cell - Activated':                  'B Cell',
    'B Cell - Memory':                     'B Cell',
    'B-reg':                               'B Cell',

    'Plasma Cell':                         'Plasma Cell',

    'T-reg':                               'CD4+ T Cell',
    'CD4+ Naive T Cell':                   'CD4+ T Cell',
    'CD4+ Th1 Cell':                       'CD4+ T Cell',
    'CD4+ Th2 Cell':                       'CD4+ T Cell',
    'CD4+ Th17 Cell':                      'CD4+ T Cell',
    'CD4+ Th22 Cell':                      'CD4+ T Cell',
    'CD4+ Memory T Cell':                  'CD4+ T Cell',

    'Double Positive CD4+CD8+ T Cell':     'CD8+ T Cell',
    'CD8+ Naive T Cell':                   'CD8+ T Cell',
    'CD8+ Effector T Cell':                'CD8+ T Cell',
    'CD8+ Memory T Cell':                  'CD8+ T Cell',
    'CD8+ Exhausted T Cell':               'CD8+ T Cell',
    'CD8+ Tissue-Resident Memory T Cell':  'CD8+ T Cell',
    'CD8+ Terminal Effector T Cell':       'CD8+ T Cell',
    'γδ T Cell (Vδ1)':                     'CD8+ T Cell',
    'Ambiguous T Cell':                    'CD8+ T Cell',  # assuming default fallback

    'NK Cell':                             'NK Cell',

    'Neutrophil - N0':                     'Neutrophil',
    'Neutrophil - N1':                     'Neutrophil',
    'Neutrophil - N2':                     'Neutrophil',

    'Dendritic Cell - cDC1':               'Dendritic Cell',
    'Dendritic Cell - cDC2':               'Dendritic Cell',
    'Dendritic Cell - pDC':                'Dendritic Cell',

    'Endothelial Cell- Vascular':          'Endothelial Cell',
    'Endothelial Cell - Lymphatic':        'Endothelial Cell',
    'Endothelial Cell - Tumor Associated ': 'Endothelial Cell',

    'Malignant Cell - Pit Like':           'Malignant Cell - Epithelial',
    'Malignant Cell - Acinar-like':        'Malignant Cell - Epithelial',
    'Malignant Cell - Epithelial':         'Malignant Cell - Epithelial',

    'Malignant Cell - EMT':                'Malignant Cell - EMT',
    'Malignant Cell - Hypoxia':            'Malignant Cell - Epithelial',
    'Malignant Cell - Mesenchymal':        'Malignant Cell - Mesenchymal',
    'Malignant Cell - Highly Proliferative':'Malignant Cell - EMT',
    'Malignant Cell - Highly Invasive':    'Malignant Cell - EMT',
    'Malignant Cell - Senescence':         'Malignant Cell - Epithelial',
    'Malignant Cell - Apoptotic':          'Malignant Cell - Epithelial',

    'Fibroblast':                          'Fibroblast',
    'myCAF':                               'CAF',
    'iCAF':                                'CAF',
    'apCAF':                               'CAF',

    'Acinar Cell':                         'Acinar Cell',
    'Acinar (REG+) Cell':                  'Acinar Cell',
    'Acinar Idling Cell':                  'Acinar Cell',

    'ADM Cell':                            'ADM Cell',

    'Ductal Cell':                         'Ductal Cell',

    'Alpha Cell':                          'Alpha Cell',
    'Beta Cell':                           'Beta Cell',
    'Gamma Cell':                          'Gamma Cell',
    'Delta Cell':                          'Delta Cell',
    'Epsilon Cell':                        'Epsilon Cell',

    'Adypocyte':                           'Adypocyte',
}
adata.obs['Level_3_final'] = adata.obs['Level_4_final'].map(level4_to_level3)

In [ ]:
level3_to_level2 = {
    'Macrophage':                  'Macrophage',
    'Monocyte':                    'Monocyte',
    'B Cell':                      'B Cell',
    'Plasma Cell':                 'B Cell',

    'CD4+ T Cell':                 'T Cell',
    'CD8+ T Cell':                 'T Cell',

    'NK Cell':                     'NK Cell',

    'Neutrophil':                  'Neutrophil',

    'Dendritic Cell':              'Dendritic Cell',

    'Endothelial Cell':            'Endothelial Cell',

    'Fibroblast':                  'Fibroblast',
    'CAF':                         'Fibroblast',

    'Acinar Cell':                 'Exocrine Cell',
    'ADM Cell':                    'Exocrine Cell',
    'Ductal Cell':                 'Exocrine Cell',

    'Malignant Cell - Epithelial': 'Malignant Cell',
    'Malignant Cell - EMT':        'Malignant Cell',
    'Malignant Cell - Mesenchymal':'Malignant Cell',

    'Alpha Cell':                  'Endocrine Cell',
    'Beta Cell':                   'Endocrine Cell',
    'Gamma Cell':                  'Endocrine Cell',
    'Delta Cell':                  'Endocrine Cell',
    'Epsilon Cell':                'Endocrine Cell',

    'Adypocyte':                   'Adypocyte',
}
adata.obs['Level_2_final'] = adata.obs['Level_3_final'].map(level3_to_level2)

In [ ]:
level2_to_level1 = {
    # Immune cells
    'Macrophage':         'Immune Cell',
    'Monocyte':           'Immune Cell',
    'B Cell':             'Immune Cell',
    'T Cell':             'Immune Cell',
    'NK Cell':            'Immune Cell',
    'Neutrophil':         'Immune Cell',
    'Dendritic Cell':     'Immune Cell',

    # Non-malignant epithelium
    'Acinar Cell':        'Epithelial Non Malignant Cell',
    'Endocrine Cell':     'Epithelial Non Malignant Cell',
    'Exocrine Cell':      'Epithelial Non Malignant Cell',
    # Malignant
    'Malignant Cell':     'Epithelial Malignant Cell',

    # Stromal
    'Endothelial Cell':   'Stromal Cell',
    'Fibroblast':         'Stromal Cell',
    'Adypocyte':          'Stromal Cell',
}
adata.obs['Level_1_final'] = adata.obs['Level_2_final'].map(level2_to_level1)

In [ ]:
adata.write_h5ad('/mnt/storage/Daniele/atlases/mouse/15_mouse_final_integration.h5ad')

In [ ]:
sc.pl.umap(adata, color = 'Level_2_final', add_outline = True)